<a href="https://colab.research.google.com/github/andrewpark19/4thyear-FYP/blob/main/randomforest_binary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow import keras
import matplotlib.pyplot as plt
import glob
import shutil
import os
import pandas as pd

In [ ]:
!unzip /content/drive/MyDrive/melanoma_data/isic_2020/train_2020/train_2020.zip > /dev/null
!unzip /content/drive/MyDrive/melanoma_data/isic_2020/test_2020/test_2020.zip > /dev/null
!unzip /content/drive/MyDrive/melanoma_data/isic_2019/train_2019/train_2019.zip > /dev/null


In [ ]:
train2020_dir = '/content/train_2020'
test2020_dir = '/content/test_2020'
train2019_dir = '/content/train_2019' 
data_dir = '/content/drive/MyDrive/melanoma_data'

def get_df(data_dir, train2020_dir, test2020_dir, train2019_dir, use_meta=False):

    ###Training Data
    #2020 Data
    df_train = pd.read_csv(os.path.join(data_dir, r'isic_2020','train_2020.csv')) #path to train_2020.csv
    df_train['filepath'] = df_train['image_name'].apply(lambda x: os.path.join(train2020_dir, f'{x}.jpg'))

    # #2019 data
    df_train2 = pd.read_csv(os.path.join(data_dir, r'isic_2019', 'train_2019.csv')) #path to train_2019.csv
    df_train2['filepath'] = df_train2['image_name'].apply(lambda x: os.path.join(train2019_dir, f'{x}.jpg'))

    # #Change Labels to match
    df_train['diagnosis']  = df_train['diagnosis'].apply(lambda x: x.replace('seborrheic keratosis', 'BKL'))
    df_train['diagnosis']  = df_train['diagnosis'].apply(lambda x: x.replace('lichenoid keratosis', 'BKL'))
    df_train['diagnosis']  = df_train['diagnosis'].apply(lambda x: x.replace('solar lentigo', 'BKL'))
    df_train['diagnosis']  = df_train['diagnosis'].apply(lambda x: x.replace('lentigo NOS', 'BKL'))
    df_train['diagnosis']  = df_train['diagnosis'].apply(lambda x: x.replace('cafe-au-lait macule', 'unknown'))
    df_train['diagnosis']  = df_train['diagnosis'].apply(lambda x: x.replace('atypical melanocytic proliferation', 'unknown'))

    df_train2['diagnosis'] = df_train2['diagnosis'].apply(lambda x: x.replace('NV', 'nevus'))
    df_train2['diagnosis'] = df_train2['diagnosis'].apply(lambda x: x.replace('MEL', 'melanoma'))

    df_train = pd.concat([df_train, df_train2]).reset_index(drop=True)

    # ###Test Data
    df_test = pd.read_csv(os.path.join(data_dir,r'isic_2020', 'test_2020.csv'))
    df_test['filepath'] = df_test['image_name'].apply(lambda x: os.path.join(test2020_dir, f'{x}.jpg'))

    if use_meta:
        df_train, df_test, meta_features, n_meta_features = get_meta_data(df_train, df_test)
    else:
        meta_features = None
        n_meta_features = 0

    diagnosis2idx = {d: idx for idx, d in enumerate(sorted(df_train.diagnosis.unique()))}
    df_train['target'] = df_train['diagnosis'].map(diagnosis2idx)
    mel_idx = diagnosis2idx['melanoma']

    return df_train, df_test, meta_features, n_meta_features, mel_idx
  

df_train, df_test, _, _, mel_idx = get_df(data_dir, train2020_dir, test2020_dir, train2019_dir, use_meta=False)

In [ ]:
df_train.loc[df_train.diagnosis == 'BCC', 'benign_malignant'] = 'malignant' 
df_train.loc[df_train.diagnosis == 'SCC', 'benign_malignant'] = 'malignant' 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(horizontal_flip=True,
                                   vertical_flip=True,
                                   rescale=1/255.0,
                                   validation_split = 0.3
                                   )

In [ ]:
SEED = 42

num_classes = 2
input_shape = (72, 72, 3)

learning_rate = 0.001
weight_decay = 0.0001
batch_size = 32
patience= 5
num_epochs = 40
image_size = 72  # We'll resize input images to this size

In [ ]:
generator = train_datagen.flow_from_dataframe(     
    dataframe=df_train,  
    directory=None,
    x_col='filepath', # name of col in data frame that contains file names
    y_col="benign_malignant", # name of col with labels 
    batch_size=42997,
    shuffle=True,
    #save_to_dir=saveDir,
    target_size=(image_size, image_size),
    color_mode='rgb',
    class_mode='sparse'
)


imgs,labels = next(generator)
print(f"imgs.shape:{imgs.shape},labels.shape:{labels.shape}")


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 15460 invalid image filename(s) in x_col="filepath". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 42997 validated image filenames belonging to 2 classes.
imgs.shape:(42997, 72, 72, 3),labels.shape:(42997,)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(imgs,labels, test_size=0.25,stratify=labels,random_state=SEED)
print(f"X_train.shape:{X_train.shape},X_val.shape:{X_val.shape}")
print(f"y_train.shape:{y_train.shape},y_val.shape:{y_val.shape}")

X_train.shape:(32247, 72, 72, 3),X_val.shape:(10750, 72, 72, 3)
y_train.shape:(32247,),y_val.shape:(10750,)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score,confusion_matrix,classification_report, accuracy_score, f1_score


nsamples, nx, ny, nrgb = X_train.shape
X_train2 = X_train.reshape((nsamples,nx*ny*nrgb))

nsamples, nx, ny, nrgb = X_val.shape
X_val2 = X_val.reshape((nsamples,nx*ny*nrgb))

model=RandomForestClassifier()
model.fit(X_train2, y_train)

y_pred=model.predict(X_val2)


In [ ]:
from sklearn.metrics import average_precision_score,classification_report, accuracy_score, precision_recall_curve, roc_auc_score, auc
print('AP:', average_precision_score(y_pred,y_val))
print('ROC:', roc_auc_score(y_pred, y_val))

precision, recall, thresholds = precision_recall_curve(y_pred, y_val)
print('AUC: ', auc(recall, precision))
print('Accuracy:', accuracy_score(y_pred,y_val))
print(classification_report(y_pred,y_val))

AP: 0.26546766369527725
ROC: 0.7909257732507113
AUC:  0.5330906561116886
Accuracy: 0.8718139534883721
              precision    recall  f1-score   support

         0.0       0.97      0.89      0.93      9895
         1.0       0.35      0.69      0.46       855

    accuracy                           0.87     10750
   macro avg       0.66      0.79      0.70     10750
weighted avg       0.92      0.87      0.89     10750

